# Summary

Common notebook for reusable components (classes, functions, etc.) that can be reused in other notebooks.

In [ ]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchmetrics

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
CONFIG = {
    "seed": 42,
    "epochs": 16,
    "img_size": 96,
    "crop_size": 32,
    "model_name": "tf_efficientnet_b0",
    "weights_name": None, # None means no pretrained weights
    "num_classes": 1,
    "train_batch_size": 32,
    "valid_batch_size": 32,
    "tta_size": 4,
    "learning_rate": 1e-4,
    
    "scheduler": "CyclicLR",
    "cl_base_lr": 1e-6,
    "cl_max_lr": 1e-4,
    "cl_step_size": None, # for both up and down, to be calculated
    
    #"scheduler": 'CosineAnnealingLR',
    #"min_lr": 1e-6,
    #"T_max": 500,
    
    "weight_decay": 1e-6,
    "train_size": 8000, # To have less training times, we do not want to train all for the baseline
    "val_size" : 200,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}


IS_INTERACTIVE = ('runtime' in get_ipython().config.IPKernelApp.connection_file)

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

## Dataset class

In [ ]:
class HCDDataset(Dataset):
    def __init__(self, df, transforms=None, transformed_images_per_item=1):
        """
        transformed_images_per_item: how many transformed images we want to return in each item
            In training it should always rreturn 1
            In validation, depends on how many we want to use in TTA
        """
        self.file_names = df['file_path'].values
        self.labels = df['label'].values
        self.transforms = transforms
        self.transformed_images_per_item = transformed_images_per_item
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        imgs = []
        if self.transforms:
            for i in range(self.transformed_images_per_item):
                imgs.append(self.transforms(image=img)["image"])
        else:
            imgs = [img]

        return {
            'image': torch.stack(imgs, dim=0),
            'label': torch.tensor(label, dtype=torch.float)
        }

## Augmentations

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Flip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.Transpose(),
        #A.RGBShift(),
        
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.RandomGamma(p=0.5),
        A.CLAHE(p=0.5),
        A.ChannelShuffle(p=0.5),
        #A.Blur(),
        A.GaussNoise(),
        #A.ElasticTransform(),
        A.Normalize(p=1),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Flip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Transpose(),
        A.Normalize(p=1),
        ToTensorV2()], p=1.)
}

## Model

In [ ]:
def load_weights(model, weights_path, device=CONFIG["device"]):
    if os.path.isfile(weights_path):
        model.load_state_dict(torch.load(weights_path, map_location=device))
        print("Loaded weights from", weights_path)
        return True
    else:
        print("No previous weights available at", weights_path)
        return False

In [ ]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, pool1, pool2):
        super(AdaptiveConcatPool2d, self).__init__()
        
        self.pool1 = pool1
        self.pool2 = pool2
    
    def forward(self, x):
        x1 = self.pool1(x)
        x2 = self.pool2(x)
        return torch.cat((x1, x2), dim=1)

In [ ]:
class HCDModel_Res50(nn.Module):
    def __init__(self, num_classes=1, weights=None, dropout_p=0.5):
        super(HCDModel_Res50, self).__init__()
        
        self.cnn = torchvision.models.resnet50(weights=weights)
        in_features = self.cnn.fc.in_features * 2 # *2 because of contact pooling layers
        self.cnn.fc = nn.Identity() # we will add dense layers manually out side of resnet
        self.cnn.avgpool = AdaptiveConcatPool2d(nn.AdaptiveAvgPool2d((1, 1)), nn.AdaptiveMaxPool2d((1, 1)))
        
        print("in_features", in_features)
        self.bn1 = nn.BatchNorm1d(in_features)
        self.linear1 = nn.Linear(in_features, 1024)
        
        self.bn2 = nn.BatchNorm1d(1024)
        self.linear2 = nn.Linear(1024, 512)
        
        self.bn3 = nn.BatchNorm1d(512)
        self.linear3 = nn.Linear(512, num_classes)
          
        self.model = nn.Sequential(
            self.cnn,
            self.bn1,
            self.linear1,
            nn.ReLU(),
            nn.Dropout(dropout_p),
            self.bn2,
            self.linear2,
            nn.ReLU(),
            nn.Dropout(dropout_p),
            self.bn3,
            self.linear3,
            nn.Sigmoid(),
        )
    
    def forward(self, images):
        return self.model(images)

## Loss function

## Training and validation functions

In [ ]:
def train_one_epoch(model, optimizer, scheduler, train_dataloader, val_dataloader, device, epoch, val_iterations=None, disable_progress_bar=False):
    
    
    dataset_size = 0
    running_loss = 0.0
    
    b_acc = torchmetrics.classification.BinaryAccuracy().to(device)
    b_auroc = torchmetrics.classification.BinaryAUROC().to(device)
    
    bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), disable=disable_progress_bar)
    
    total_step = len(train_dataloader)
    log_step = math.ceil(total_step / 10) # we log each epoch 10 times in log

    epoch_start_time = time.time()
    
    # If not set, do 1 validation per epoch
    if val_iterations is None:
        val_iterations = len(train_dataloader)
        
    history = defaultdict(list)
    best_valid_auroc = -np.inf
    best_valid_loss = np.inf
    best_valid_model_wts = None
    
    for step, data in bar:
        model.train()
        optimizer.zero_grad()
        
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.float)
        
        # Need to squeeze the 1 dim due to support of TTA in val
        images = torch.squeeze(images)
        batch_size = images.size(0)
        
        outputs = torch.squeeze(model(images))
        
        loss = torch.nn.BCELoss()(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        if not CONFIG["scheduler"] == "ReduceLROnPlateau":
            scheduler.step()
            
        #_, predicted = torch.max(outputs, 1)
        predicted = outputs
        
        b_acc(predicted, labels)
        train_acc = b_acc.compute().item()
        
        b_auroc(predicted, labels)
        train_auroc = b_auroc.compute().item()
        
        running_loss += (loss.item() * batch_size)

        dataset_size += batch_size
        
        train_loss = running_loss / dataset_size
        
        epoch_time = round(time.time() - epoch_start_time)
        bar.set_postfix(Epoch=epoch, Epoch_Time=epoch_time, Train_Loss=train_loss, Train_Acc=train_acc,Train_AUROC=train_auroc,
                        LR=f"{optimizer.param_groups[0]['lr']}~{optimizer.param_groups[-1]['lr']}")
        
        if disable_progress_bar and (step%log_step==0):
            print(step, "/", total_step)
        

    
        if (step+1) % val_iterations == 0:
            val_loss, val_acc, val_auroc = valid(
                model, val_dataloader, device=CONFIG['device'],
                disable_progress_bar=(not IS_INTERACTIVE)
            )
             
            history['Train Loss'].append(train_loss)
            history['Valid Loss'].append(val_loss)
            history['Train Accuracy'].append(train_acc)
            history['Valid Accuracy'].append(val_acc)
            history['Train AUROC'].append(train_auroc)
            history['Valid AUROC'].append(val_auroc)
            #history['lr'].append(scheduler.get_lr()[0])
            
            if  val_loss < best_valid_loss:
                best_valid_auroc = val_auroc
                best_valid_loss = val_loss
                best_valid_model_wts = copy.deepcopy(model.state_dict())
    
    if CONFIG["scheduler"] == "ReduceLROnPlateau":
        scheduler.step(best_valid_loss)

    if disable_progress_bar:
        print(f"Epoch={epoch}, Epoch_Time={epoch_time}, Train_Loss={train_loss}, Train_Acc={train_acc}, Train_AUROC={train_auroc},LR={optimizer.param_groups[0]['lr']}~{optimizer.param_groups[-1]['lr']}")

    
    gc.collect()
    
    return best_valid_auroc, best_valid_loss, best_valid_model_wts, history

@torch.inference_mode()
def valid(model, dataloader, device, disable_progress_bar=False):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    b_acc = torchmetrics.classification.BinaryAccuracy().to(device)
    b_auroc = torchmetrics.classification.BinaryAUROC().to(device)
    
    for step, data in enumerate(dataloader): 
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        # Process TTA by splitting the input image tensor
        # split along the dim=1, which is multiple transformed images for TTA
        images = torch.chunk(images, chunks=images.size(1), dim=1)
        
        outputs = []
        
        for image in images:
            outputs.append(torch.squeeze(model(torch.squeeze(image))))
        outputs = torch.sum(torch.stack(outputs), dim=0) / len(images)
        

        loss = torch.nn.BCELoss()(outputs, labels)

        #_, predicted = torch.max(outputs, 1)
        predicted = outputs
        acc = torch.sum( predicted == labels )
        
        b_acc(predicted, labels)
        epoch_acc = b_acc.compute().item()
        
        b_auroc(predicted, labels)
        epoch_auroc = b_auroc.compute().item()
        
        running_loss += (loss.item() * batch_size)        
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        
    print(f"Valid_Loss={epoch_loss}, Valid_Acc={epoch_acc}, Valid_AUROC={epoch_auroc}")
        
    
    gc.collect()
    
    return epoch_loss, epoch_acc, epoch_auroc


## Run training function

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs, train_dataloader, val_dataloader, k_fold,
                 val_iterations=None, 
                 stop_epochs=4, # If val_loss stops improve for such epochs, stop the training
                 save_all_epoch=False, # If models for each epoch regardless of performance should be saved
                ):
    print("Run training for folder", k_fold)
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    best_epoch_auroc = -np.inf
    history = defaultdict(list)
    
    stop = 0
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        valid_auroc, valid_loss, valid_model_wts, epoch_history = train_one_epoch(
            model, optimizer, scheduler, 
            train_dataloader=train_dataloader, 
            val_dataloader=val_dataloader,
            device=CONFIG['device'], epoch=epoch,
            disable_progress_bar=(not IS_INTERACTIVE),
            val_iterations=val_iterations,
        )
    
        history['Train Loss'].extend(epoch_history['Train Loss'])
        history['Valid Loss'].extend(epoch_history['Valid Loss'])
        history['Train Accuracy'].extend(epoch_history['Train Accuracy'])
        history['Valid Accuracy'].extend(epoch_history['Valid Accuracy'])
        history['Train AUROC'].extend(epoch_history['Train AUROC'])
        history['Valid AUROC'].extend(epoch_history['Valid AUROC'])
        #history['lr'].extend(epoch_history['lr'])
        
        # deep copy the model
        improved = False
        if valid_loss < best_epoch_loss:
            improved = True
            print(f"{b_}Validation loss improved ({best_epoch_loss} ---> {valid_loss})")
            best_epoch_loss = valid_loss
            best_epoch_auroc = valid_auroc
            stop = 0 # loss improved, reset stop
        else:
            stop +=1
            
        if improved or save_all_epoch:
            PATH = "AUROC{:.2f}_Loss{:.4f}_fold{:.0f}_epoch{:.0f}.bin".format(valid_auroc, valid_loss, k_fold, epoch)
            torch.save(valid_model_wts, PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        if stop >= stop_epochs:
            print(f"Performance havn't improve for {stop} epochs, stop training!")
            break
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    print("-----------------------------------------------------")
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history, best_epoch_loss, best_epoch_auroc

In [ ]:
def get_optim_params(model):
    """
    Config different lr for different layers, the deeper the smaller
    """ 
    return [
        {'params': model.cnn.conv1.parameters(), 'lr': CONFIG["learning_rate"]/1000},
        {'params': model.cnn.bn1.parameters(), 'lr': CONFIG["learning_rate"]/1000},
        {'params': model.cnn.layer1.parameters(), 'lr': CONFIG["learning_rate"]/100},
        {'params': model.cnn.layer2.parameters(), 'lr': CONFIG["learning_rate"]/100},
        {'params': model.cnn.layer3.parameters(), 'lr': CONFIG["learning_rate"]/10},
        {'params': model.cnn.layer4.parameters(), 'lr': CONFIG["learning_rate"]/10},
        {'params': model.bn1.parameters(), 'lr': CONFIG["learning_rate"]/100},
        {'params': model.bn2.parameters(), 'lr': CONFIG["learning_rate"]/100},
        {'params': model.bn3.parameters(), 'lr': CONFIG["learning_rate"]/100},
        {'params': model.linear1.parameters(), 'lr': CONFIG["learning_rate"]},
        {'params': model.linear2.parameters(), 'lr': CONFIG["learning_rate"]},
        {'params': model.linear3.parameters(), 'lr': CONFIG["learning_rate"]},
    ]

In [ ]:
def fetch_scheduler_optimizer(model_parameters):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        optimizer = optim.Adam(model_parameters, lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr'])
        
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        optimizer = optim.Adam(model_parameters, lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CyclicLR':
        optimizer = optim.SGD(model_parameters, lr=CONFIG['learning_rate'], momentum=0.9)
        scheduler = lr_scheduler.CyclicLR(optimizer,
                                          base_lr=CONFIG['cl_base_lr'], 
                                          max_lr=CONFIG['cl_max_lr'],
                                          step_size_up=CONFIG['cl_step_size'],
                                         )
    elif CONFIG['scheduler'] == 'ReduceLROnPlateau':
        optimizer = optim.SGD(model_parameters, lr=CONFIG['learning_rate'], momentum=0.9)
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, min_lr=CONFIG["min_lr"],
                                                   patience=CONFIG['rlrop_patience'], threshold=CONFIG['rlrop_thr'], verbose=True)
    elif CONFIG['scheduler'] == None:
        return None, None
        
    return scheduler, optimizer

## Prepare loaders

In [ ]:
def prepare_loaders(train_df, val_df, tta_size=1):
    
    train_dataset = HCDDataset(train_df, transforms=data_transforms["train"])
    valid_dataset = HCDDataset(val_df, transforms=data_transforms["valid"], transformed_images_per_item=tta_size)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

# Plot history

In [ ]:
def plot_history(history, metric_name):
    plt.plot( range(history.shape[0]), history[f"Train {metric_name}"].values, label=f"Train {metric_name}")
    plt.plot( range(history.shape[0]), history[f"Valid {metric_name}"].values, label=f"Valid {metric_name}")
    plt.xlabel("Probs")
    plt.ylabel(metric_name)
    plt.grid()
    plt.legend()
    plt.show()

## Generate submit csv

In [ ]:
def generate_submit_csv(models, root_folder, transforms):
    test_df = pd.read_csv(os.path.join(root_folder, "sample_submission.csv"))
    test_df["file_path"] = test_df["id"].apply(lambda image_id: os.path.join(root_folder, "test", f"{image_id}.tif"))
    print("test_df shape:", test_df.shape)

    test_dataset = HCDDataset(test_df, transforms=transforms["valid"], transformed_images_per_item=CONFIG["tta_size"])
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['valid_batch_size'], 
                                  num_workers=2, shuffle=False, pin_memory=True)

    preds = []

    total_step = len(test_loader)
    log_step = math.ceil(total_step / 10) # we log each epoch 10 times in log

    with torch.no_grad():
        bar = tqdm(enumerate(test_loader), total=len(test_loader), disable=(not IS_INTERACTIVE))
        for step, data in bar:        
            images = data['image'].to(CONFIG["device"], dtype=torch.float)        
            
            outputs = []
            for model in models:
                
                # Process TTA by splitting the input image tensor
                # split along the dim=1, which is multiple transformed images for TTA
                images = torch.chunk(images, chunks=images.size(1), dim=1)

                for image in images:
                    outputs.append(model.sigmoid(model(torch.squeeze(image))))
 
            _, predicted = torch.max(torch.sum(torch.stack(outputs), dim=0), dim=1)
            
            preds.extend(predicted.tolist())

            if not IS_INTERACTIVE and (step%log_step==0):
                print(step, "/", total_step)

    print(len(preds))
    
    test_df["label"] = preds
    test_df[["id", "label"]].to_csv("submission.csv", index=False)